# Figure 5: Basin of Attraction for Textured Motif Registration

The code below tests the with-background-modeling registration optimization formulation, i.e. equation (4) in the paper (arXiv v1), with either a euclidean or similarity motion model.

The implementation is in numpy.

In [ ]:
# Imports
import sys
sys.path.append("../src")
from registration_np import (generate_data, dilate_support, registration_l2_exp, get_default_pgd_dict)

In [ ]:
# Change `mode` to 'euclidean' below to run the experiment for SE(2) motion model
tra_vals = np.linspace(-30, 30, 9)
sca_vals = np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 2.5, 3, 3.5, 4])
tra_list = np.repeat(tra_vals, len(sca_vals))
sca_list = np.tile(sca_vals, len(tra_vals))
num = len(tra_list)

N_try = 10


mode = 'similarity'
corr_lst = np.zeros((N_try, num))

for i_try in range(N_try):
    print()
    print('===== {} ====='.format(i_try))
    
    for idx in range(num):
        
        if mode == 'similarity':
            dil = 1
            phi = 0
            b = np.zeros((2,))
            optim_vars = [dil, phi, b]
        elif mode == 'euclidean':
            phi = 0
            b = np.zeros((2,))
            optim_vars = [phi, b]

        sigma = 10
        
        tra = tra_list[idx]
        sca = sca_list[idx]
        # Convert to affine tranform expression: A and b
        A = sca * np.eye(2)
        b = np.array([tra, 0])
        X0, X, Om, c = generate_data(mode, A, b)

        for repeat in range(5):
            Om_tilde = dilate_support(Om,sigma)
            param_dict = get_default_pgd_dict(
                step=0.005*sigma, max_iter=50, epoch_len=10, parametric=True, center=c, tol=1e-5,
                sigma=sigma, quiet=False, use_nesterov=False )
            tau_u, tau_v, optim_vars, error, Rvals = registration_l2_exp( X, X0, Om_tilde, Om, c,
                                                                       mode, optim_vars, param_dict=param_dict,
                                                                       visualize=False)
            sigma /= 2

        Rval = np.max(Rvals)
        corr_lst[i_try, idx] = Rval
        print([idx, Rval])
        
        np.save('gridsearch.npy', corr_lst)
